### Sample dashboard to Visualize Seasonal Forecast Data -  JRC

#### Run the below line to launch the GUI..

In [ ]:
exec("""\nfrom ipyleaflet import Map, DrawControl, GeoJSON, LayersControl, SearchControl, TileLayer\nfrom ipywidgets import widgets, Layout\nfrom IPython.display import display, HTML\nfrom datetime import datetime\nimport dateutil.relativedelta\nfrom apps.productmanagement.extended_Dataset import ExtDataset\nimport xarray as xr\ndmax = datetime.today()\ndmin = dmax + dateutil.relativedelta.relativedelta(months=-3)\n\ndatemax= widgets.DatePicker(value=dmax,description='Max date:',\n                             style={'description_width': 'initial'},indent=False,layout=widgets.Layout(width='200px', height='32px'))\ndatemin = widgets.DatePicker(value=dmin,description='Min date:',\n                             style={'description_width': 'initial'},indent=False,layout=widgets.Layout(width='200px', height='32px'))\ndropdown = widgets.Dropdown(description='Products:', options=['s51-monthly-tp','s51-monthly-2mt','s51-monthly-sst'])\nbutton = widgets.Button(\n    description='List the files',\n    disabled=False,\n    button_style='', # 'success', 'info', 'warning', 'danger' or ''\n    tooltip='List the files for the selected product and dates',\n    icon='check' # (FontAwesome names without the `fa-` prefix)\n)\n\nproducts = widgets.Select(description='Files', options=[],rows=20, layout=Layout(width='700px', min_width='200px', height='108px'))\n\nclear_button = widgets.Button(\n    description='Clear List',\n    disabled=False,\n    button_style='', # 'success', 'info', 'warning', 'danger' or ''\n    tooltip='Clear the output in the display',\n    icon='check' # (FontAwesome names without the `fa-` prefix)\n)\n\nlist_dimension_button = widgets.Button(\n    description='List Dimenions',\n    disabled=False,\n    button_style='', # 'success', 'info', 'warning', 'danger' or ''\n    tooltip='List Dimenions',\n    icon='check' # (FontAwesome names without the `fa-` prefix)\n)\n\noutput = widgets.Output()\nDimension_output = widgets.Output()\nview_output = widgets.Output()\ndisplay(datemin,datemax,dropdown, button, products,output, Dimension_output,view_output)\n# kwargs = {\n#         'product_code': 's5-monthly-tp',\n#         'version': "1.0",\n#         'sub_product_code': 'tp',\n#         'mapset': 'CDS-ACP-1deg'\n#     }\n# tp = ExtDataset(**kwargs)\n\ntime = widgets.Dropdown(description='Forecast time', options=[])\nnumber = widgets.Dropdown(description='Ensemble member',options=[])\n\ndef on_button_clicked(b):\n    product= dropdown.value\n    sprod = dropdown.value.split("-")[-1]\n    kwargs = {\n            'product_code': product,\n            'version': "1.0",\n            'sub_product_code': sprod,\n            'mapset': 'CDS-ACP-1deg'\n        }\n    tp = ExtDataset(**kwargs)\n    dmin = datemin.value.strftime('%Y%m%d')\n    dmax = datemax.value.strftime('%Y%m%d')\n    tp.filter(str(dmin),str(dmax))\n    files = tp.get_filenames_range()\n    products.options = files\n    #display(clear_button)\n    with output:\n        #print(dmin+"-->"+dmax+ dropdown.value)\n        #print(list(files))\n        output.clear_output()\n        display(clear_button, list_dimension_button)\nbutton.on_click(on_button_clicked)\n\ndef on_button_clear(b):\n    products.options = []\n    with output:\n        Dimension_output.clear_output()\n        \nclear_button.on_click(on_button_clear)\n\nview_button = widgets.Button(\n    description='View data',\n    disabled=False,\n    button_style='', # 'success', 'info', 'warning', 'danger' or ''\n    tooltip='View data',\n    icon='check' # (FontAwesome names without the `fa-` prefix)\n)\n\ndef on_list_dimension(b):\n    time.options=[]\n    number.options=[]\n    product= dropdown.value\n    sprod = dropdown.value.split("-")[-1]\n    kwargs = {\n            'product_code': product,\n            'version': "1.0",\n            'sub_product_code': sprod,\n            'mapset': 'CDS-ACP-1deg'\n        }\n    extD = ExtDataset(**kwargs)\n    extD.assign_dims_values(products.value)\n    # time = widgets.Dropdown(options=tp.dimensions.get('time').value)\n    # number = widgets.Dropdown(options=tp.dimensions.get('number').value)\n    time.options=extD.dimensions.get('time').value\n    number.options=extD.dimensions.get('number').value\n    \n    with Dimension_output:\n        print(b)\n        Dimension_output.clear_output()\n        display(time,number, view_button)\n\nlist_dimension_button.on_click(on_list_dimension)\n\ndef on_view(b):\n    product= dropdown.value\n    sprod = dropdown.value.split("-")[-1]\n    kwargs = {\n            'product_code': product,\n            'version': "1.0",\n            'sub_product_code': sprod,\n            'mapset': 'CDS-ACP-1deg'\n        }\n    extD = ExtDataset(**kwargs)\n    # extD.view_nc(products.value)\n    with view_output:\n        view_output.clear_output()\n        ds = xr.open_dataset(products.value)\n        ds[extD.data_variable].sel(time=time.value,number=number.value).plot(figsize=(36,6))\n        # display(extD.view_nc(products.value))\n\nview_button.on_click(on_view)\n""")